In [2]:
import pandas as pd
import numpy as np

In [70]:
df = pd.read_csv('../data/population.csv')

In [71]:
columns = df.columns.tolist()
# fix column years format
for i in range(4, len(columns)):
	columns[i] = columns[i][:4]
df.columns = columns

In [72]:
attributes_dic = {
    "life_expectancy_male": "SP.DYN.LE00.MA.IN",
    "life_expectancy_female": "SP.DYN.LE00.FE.IN",
    "birth_rate": "SP.DYN.CBRT.IN",
    "death_rate": "SP.DYN.CDRT.IN",
    "growth_rate": "SP.POP.GROW",
    "fertility_rate": "SP.DYN.TFRT.IN",
    "total_population": "SP.POP.TOTL",
    "urban_population": "SP.URB.TOTL",
    "rural_population": "SP.RUR.TOTL",
    "population_ages_0_to_14": "SP.POP.0014.TO",
    "population_ages_15_to_64": "SP.POP.1564.TO",
    "population_ages_65_and_above": "SP.POP.65UP.TO",
}

In [73]:
def transpose_attributes(dataframe, attribute):
	dataframe.drop(columns=["Series Name", "Series Code", "Country Name"], inplace=True)
	columns = dataframe.columns.tolist()
	df.melt(id_vars="Country Code", value_vars=columns[1:], var_name="Year", value_name=attribute)

In [79]:
attributes_df_dic = {}
for key, value in attributes_dic.items(): 
	filt = (df["Series Code"] == value)
	attribute_df = df.loc[filt].copy()
	transpose_attributes(attribute_df, key)
	attributes_df_dic[key] = attribute_df 

In [ ]:
attributes_lst_dic = list(attributes_df_dic.values())
population_df = attributes_lst_dic[0] 
for i in range(1, len(attributes_lst_dic)):
	population_df = pd.merge(population_df, attributes_lst_dic[i], how="left", left_on=["Country Code", "Year"], right_on=["Country Code", "Year"])